# Cross validation: An Ensemble of Facebook's Prophet, Regression with ARIMA errors and Holt-Winters
---

An unweighted ensemble of 

* Facebook Prophet with New Year holiday;
* Regression with new year holiday and (auto) ARIMA errors; and 
* Holt-Winters Exponential Smoothing (statespace formulation)

The ARIMA process modelled is a (1,1,3)(1,0,1,7).  This was determined by auto_arima from the `pmdarima` package.

Note that HW provides now way to easily incorporate special events.

This notebook conducts cross validation of the method using a rolling forecast origin method.

> The Reg with ARIMA error model is based on SWAST's data.  It may be prudent for each trust to fit their own automatic ARIMA model.

**The notebook outputs:**
* MASE, RMSE and MAPE at 7 day intervals from 7 to 84 days.
* 80 and 95% prediction intervals between 7 and 84 days.

These are saved into the folder `results/model_selection/stage1/`

---

# Imports

In [1]:
import pandas as pd
import numpy as np

#error measures
from forecast_tools.metrics import (mean_absolute_scaled_error, 
                                    root_mean_squared_error,
                                    symmetric_mean_absolute_percentage_error)

#models
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.exponential_smoothing import ExponentialSmoothing
from fbprophet import Prophet

import warnings
warnings.filterwarnings('ignore')

In [2]:
import statsmodels as sm
import pmdarima 

print(sm.__version__)
print(pmdarima.__version__)

0.11.0
1.5.2


In [3]:
#to select exceptionally busy days as covariates.
from amb_forecast.feature_engineering import (regular_busy_calender_days)

In [4]:
#custom ensemble class
from amb_forecast.ensemble import (Ensemble, UnweightedVote)

# Data Input

The constants `TOP_LEVEL`, `STAGE`, `REGION`,`TRUST` and `METHOD` are used to control data selection and the directory for outputting results.  

> Output file is `f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv'.csv`.  where metric will be smape, rmse, mase, coverage_80 and coverage_95. Note: `REGION`: is also used to select the correct data from the input dataframe.

In [5]:
TOP_LEVEL = '../../../results/model_selection'
STAGE = 'stage1'
REGION = 'Trust'
METHOD = 'fbp-arima-hw'

FILE_NAME = 'Daily_Responses_5_Years_2019_full.csv'

#split training and test data.
TEST_SPLIT_DATE = '2019-01-01'

#second subdivide: train and val
VAL_SPLIT_DATE = '2017-07-01'

#discard data after 2020 due to coronavirus
#this is the subject of a seperate study.
DISCARD_DATE = '2020-01-01'

In [6]:
#read in path
path = f'../../../data/{FILE_NAME}'

In [7]:
def pre_process_daily_data(path, index_col, by_col, 
                           values, dayfirst=False):
    '''
    Daily data is stored in long format.  Read in 
    and pivot to wide format so that there is a single 
    colmumn for each regions time series.
    '''
    df = pd.read_csv(path, index_col=index_col, parse_dates=True, 
                     dayfirst=dayfirst)
    df.columns = map(str.lower, df.columns)
    df.index.rename(str(df.index.name).lower(), inplace=True)
    
    clean_table = pd.pivot_table(df, values=values.lower(), 
                                 index=[index_col.lower()],
                                 columns=[by_col.lower()], aggfunc=np.sum)
    
    clean_table.index.freq = 'D'
    
    return clean_table

In [8]:
clean = pre_process_daily_data(path, 'Actual_dt', 'ORA', 'Actual_Value', 
                               dayfirst=False)
clean.head()

ora,BNSSG,Cornwall,Devon,Dorset,Gloucestershire,OOA,Somerset,Trust,Wiltshire
actual_dt,,,,,,,,,
2013-12-30,415.0,220.0,502.0,336.0,129.0,NaN,183.0,2042.0,255.0
2013-12-31,420.0,236.0,468.0,302.0,128.0,NaN,180.0,1996.0,260.0
2014-01-01,549.0,341.0,566.0,392.0,157.0,NaN,213.0,2570.0,351.0
2014-01-02,450.0,218.0,499.0,301.0,115.0,NaN,167.0,2013.0,258.0
2014-01-03,419.0,229.0,503.0,304.0,135.0,NaN,195.0,2056.0,269.0


## Train Test Split

In [9]:
def ts_train_test_split(data, split_date):
    '''
    Split time series into training and test data
    
    Parameters:
    -------
    data - pd.DataFrame - time series data.  Index expected as datatimeindex
    split_date - the date on which to split the time series
    
    Returns:
    --------
    tuple (len=2) 
    0. pandas.DataFrame - training dataset
    1. pandas.DataFrame - test dataset
    '''
    train = data.loc[data.index < split_date]
    test = data.loc[data.index >= split_date]
    return train, test

In [10]:
train, test = ts_train_test_split(clean, split_date=TEST_SPLIT_DATE)

#exclude data after 2020 due to coronavirus.
test, discard = ts_train_test_split(test, split_date=DISCARD_DATE)

#train split into train and validation
train, val = ts_train_test_split(train, split_date=VAL_SPLIT_DATE)


In [11]:
#amount of training data
train.shape

(1279, 9)

In [12]:
#amount of validation data
val.shape

(549, 9)

# New years day

In [13]:
exceptional = regular_busy_calender_days(train[REGION], quantile=0.99)

In [14]:
new_year = pd.DataFrame({
                         'holiday': 'new_year',
                         'ds': pd.date_range(start=exceptional[0], 
                                             periods=20, 
                                             freq='YS')
                        })

In [15]:
new_year.head()

,holiday,ds
0,new_year,2013-01-01
1,new_year,2014-01-01
2,new_year,2015-01-01
3,new_year,2016-01-01
4,new_year,2017-01-01


# Wrapper classes for Prophet, statsmodels ARIMA and statsmodels Exponential Smoothing.

Adapter/wrapper classes to enable usage within `Ensemble` class and work with cross validation.

In [16]:
class FbProphetWrapper(object):
    '''
    Facade for FBProphet object - so that it can be
    used within Ensemble with methods from other packages

    '''
    def __init__(self, training_index, holidays=None, interval_width=0.8,
                 mcmc_samples=0, changepoint_prior_scale=0.05):
        self._training_index = training_index
        self._holidays = holidays
        self._interval_width = interval_width
        self._mcmc_samples = mcmc_samples
        self._cp_prior_scale = changepoint_prior_scale

    def _get_resids(self):
        return self._train - self._forecast['yhat'][:-self._h]

    def _get_preds(self):
        return self._forecast['yhat'][:-self._h].to_numpy()

    def fit(self, train):
        
        self._model = Prophet(holidays=self._holidays, 
                              interval_width=self._interval_width,
                              mcmc_samples=self._mcmc_samples,
                              changepoint_prior_scale=self._cp_prior_scale,
                              daily_seasonality=False)
        
        
        self._model.fit(self._pre_process_training(train))
        self._t = len(train)
        self._train = train
        self.predict(len(train))

    def _pre_process_training(self, train):

        if len(train.shape) > 1:
            y_train = train[:, 0]
        else:
            y_train = train

        y_train = np.asarray(y_train)
            
        #extend the training index
        if len(y_train) > len(self._training_index):
            self._training_index = pd.date_range(start=self._training_index[0], 
                                                 periods=len(y_train),
                                                 freq=self._training_index.freq)
        
        
        prophet_train = pd.DataFrame(self._training_index)
        prophet_train['y'] = y_train
        prophet_train.columns = ['ds', 'y']
        
        return prophet_train

    def predict(self, h, return_conf_int=False, alpha=0.2):
        '''
        forecast h steps ahead.
        
        Params:
        ------
        h: int
            h-step forecast
        
        return_conf_int: bool, optional (default=False)
            return 1 - alpha PI
        
        alpha: float, optional (default=0.2)
            return 1 - alpha PI
                       
        Returns:
        -------
        np.array
            If return_conf_int = False returns preds only
            
        np.array, np.array
            If return_conf_int = True returns tuple of preds, pred_ints
        '''
        if isinstance(h, (np.ndarray, pd.DataFrame)):
            h = len(h)
        
        self._h = h
        future = self._model.make_future_dataframe(periods=h)
        self._forecast = self._model.predict(future)

        if return_conf_int:
            return (self._forecast['yhat'][-h:].to_numpy(), 
                    self._forecast[['yhat_lower', 'yhat_upper']][-h:].to_numpy())
        else:
            return self._forecast['yhat'][-h:].to_numpy()
            

    fittedvalues = property(_get_preds)
    resid = property(_get_resids)

In [17]:
class ARIMAWrapper(object):
    '''
    Facade for statsmodels.statespace
    '''
    def __init__(self, order, seasonal_order, training_index, holidays=None):
        self._order = order
        self._seasonal_order = seasonal_order
        self._training_index = training_index
        self._holidays = holidays

    def _get_resids(self):
        return self._fitted.resid

    def _get_preds(self):
        return self._fitted.fittedvalues
    
    def _encode_holidays(self, holidays, idx):
        dummy = idx.isin(holidays).astype(int)
        dummy = pd.DataFrame(dummy)
        dummy.columns = ['holiday']
        dummy.index = idx
        return dummy

    def fit(self, y_train):
        
        #extend training index
        if len(y_train) > len(self._training_index):

            self._training_index = pd.date_range(start=self._training_index[0], 
                                                 periods=len(y_train),
                                                 freq=self._training_index.freq)
            
        holiday_train = None
        if not self._holidays is None:
            holiday_train = self._encode_holidays(self._holidays, 
                                                  self._training_index)
    
        
        self._model = ARIMA(endog=y_train,
                            exog=holiday_train,
                            order=self._order, 
                            seasonal_order=self._seasonal_order)#,
                            #enforce_stationarity=False)
        
        self._fitted = self._model.fit()
        self._t = len(train)
        
    
    def predict(self, horizon, return_conf_int=False, alpha=0.2):
        '''
        forecast h steps ahead.
        
        Params:
        ------
        h: int
            h-step forecast
        
        return_conf_int: bool, optional (default=False)
            return 1 - alpha PI
        
        alpha: float, optional (default=0.2)
            return 1 - alpha PI
                       
        Returns:
        -------
        np.array
            If return_conf_int = False returns preds only
            
        np.array, np.array
            If return_conf_int = True returns tuple of preds, pred_ints
        '''
        
        #+1 to date range then trim off the first value

        f_idx = pd.date_range(start=self._training_index[-1], 
                              periods=horizon+1,
                              freq=self._training_index.freq)[1:]
        
        #encode holidays if included.
        exog_holiday = None
        if not self._holidays is None:
            exog_holiday = self._encode_holidays(self._holidays, f_idx)
        
    
        forecast = self._fitted.get_forecast(horizon, exog=exog_holiday)
        mean_forecast = forecast.summary_frame()['mean'].to_numpy()
        
        if return_conf_int:
            df = forecast.summary_frame(alpha=alpha)
            pi = df[['mean_ci_lower', 'mean_ci_upper']].to_numpy()
            return mean_forecast, pi
            
        
        else:
            return mean_forecast

    fittedvalues = property(_get_preds)
    resid = property(_get_resids)  

In [18]:
class ExponentialSmoothingWrapper:
    '''
    Facade for statsmodels exponential smoothing models.  This wrapper
    provides a common interface for all models and allow interop with
    the custom time series cross validation code.
    '''
    def __init__(self, trend=False, damped_trend=False, seasonal=None):
        self._trend = trend
        self._seasonal= seasonal
        self._damped_trend = damped_trend

    def _get_resids(self):
        return self._fitted.resid

    def _get_preds(self):
        return self._fitted.fittedvalues

    def fit(self, train):
        '''
        Fit the model
        
        Parameters:
        train: array-like
            time series to fit.
        '''
        self._model = ExponentialSmoothing(endog=train,
                                          trend=self._trend, 
                                          damped_trend=self._damped_trend,
                                          seasonal=self._seasonal)
        self._fitted = self._model.fit()
        self._t = len(train)
    
    def predict(self, horizon, return_conf_int=False, alpha=0.2):
        '''
        Forecast the time series from the final point in the fitted series.
        
        Parameters:
        ----------
        
        horizon: int
            steps ahead to forecast 
            
        return_conf_int: bool, optional (default=False)
            Return prediction interval?  
            
        alpha: float
            Used if return_conf_int=True. 100(1-alpha) interval.
        '''
        
        forecast = self._fitted.get_forecast(horizon)
        
        mean_forecast = forecast.summary_frame()['mean'].to_numpy()
        
        if return_conf_int:
            df = forecast.summary_frame(alpha=alpha)
            pi = df[['mean_ci_lower', 'mean_ci_upper']].to_numpy()
            return mean_forecast, pi        
        else:
            return mean_forecast

    fittedvalues = property(_get_preds)
    resid = property(_get_resids)

# Example of fitting the ensemble
1. Regression with New Year Holiday and Auto ARIMA errors
2. FBProphet with new years day holiday.
3. Holt-Winters Exponential Smoothing

The code below demonstrates how to fit the model.

In [19]:
model_1 = ARIMAWrapper(order=(1,1,3), seasonal_order=(1,0,1,7), 
                       training_index=train.index,
                       holidays=new_year['ds'].tolist())

model_2 = FbProphetWrapper(training_index=train.index, 
                           holidays=new_year)

model_3 = ExponentialSmoothingWrapper(trend=True, damped_trend=True, 
                                      seasonal=7)

In [20]:
estimators = {'arima': model_1, 'fbp': model_2, 'hw': model_3}
ens = Ensemble(estimators, UnweightedVote())

In [21]:
#fit to training data in chosen region
ens.fit(train[REGION])

In [22]:
#predict 7 days ahead
H = 7
ens_preds = ens.predict(horizon=H)

In [23]:
#view predictions
ens_preds

array([2275.15048284, 2259.43983053, 2157.54865967, 2088.51607366,
       2081.27992798, 2093.40228599, 2137.83112478])

In [24]:
#with prediction intervals
ens_preds, pi = ens.predict(horizon=H, return_conf_int=True)

In [25]:
ens_preds

array([2275.15048284, 2259.43983053, 2157.54865967, 2088.51607366,
       2081.27992798, 2093.40228599, 2137.83112478])

In [26]:
pi

array([[2177.77495698, 2373.50993744],
       [2159.5652313 , 2362.34867649],
       [2053.35824724, 2264.04003533],
       [1980.218854  , 2195.88773709],
       [1972.47693801, 2189.49170262],
       [1982.14001586, 2203.39939369],
       [2023.84366814, 2254.55811812]])

# Cross validation functions

`time_series_cv` implements rolling forecast origin cross validation for time series.  
It does not calculate forecast error, but instead returns the predictions, pred intervals and actuals in an array that can be passed to any forecast error function. (this is for efficiency and allows additional metrics to be calculated if needed).

In [27]:
def time_series_cv(model, train, val, horizons, alpha=0.2, step=1):
    '''
    Time series cross validation across multiple horizons for a single model.

    Incrementally adds additional training data to the model and tests
    across a provided list of forecast horizons. Note that function tests a
    model only against complete validation sets.  E.g. if horizon = 15 and 
    len(val) = 12 then no testing is done.  In the case of multiple horizons
    e.g. [7, 14, 28] then the function will use the maximum forecast horizon
    to calculate the number of iterations i.e if len(val) = 365 and step = 1
    then no. iterations = len(val) - max(horizon) = 365 - 28 = 337.
    
    Parameters:
    --------
    model - forecasting model

    train - np.array - vector of training data

    val - np.array - vector of validation data

    horizon - list of ints, forecast horizon e.g. [7, 14, 28] days
    
    alpha - float, optional (default=0.2)
        1 - alpha prediction interval specification

    step -- int, optional (default=1)
            step taken in cross validation 
            e.g. 1 in next cross validation training data includes next point 
            from the validation set.
            e.g. 7 in the next cross validation training data includes next 7 points
            (default=1)
            
    Returns:
    -------
    np.array, np.array, np.array
        - cv_preds, cv_test, cv_intervals
    '''
    
    #point forecasts
    cv_preds = [] 
    #ground truth observations
    cv_actuals = [] 
    #prediction intervals
    cv_pis = []
    
    split = 0

    print('split => ', end="")
    for i in range(0, len(val) - max(horizons) + 1, step):
        split += 1
        print(f'{split}, ', end="")
                
        train_cv = np.concatenate([train, val[:i]], axis=0)
        model.fit(train_cv)
        
        #predict the maximum horizon 
        preds, pis = model.predict(horizon=len(val[i:i+max(horizons)]), 
                                   return_conf_int=True,
                                   alpha=alpha)        
        cv_h_preds = []
        cv_test = []
        cv_h_pis = []
        
        #sub horizon calculations
        for h in horizons:
            #store the h-step prediction
            cv_h_preds.append(preds[:h])
            #store the h-step actual value
            cv_test.append(val.iloc[i:i+h])    
            cv_h_pis.append(pis[:h])
                     
        cv_preds.append(cv_h_preds)
        cv_actuals.append(cv_test)
        cv_pis.append(cv_h_pis)
        
    print('done.\n')        
    return cv_preds, cv_actuals, cv_pis

## Custom functions for calculating CV scores for point predictions and coverage.

These functions have been written to work with the output of `time_series_cv`

In [28]:
def split_cv_error(cv_preds, cv_test, error_func):
    '''
    Forecast error in the current split
    
    Params:
    -----
    cv_preds, np.array
        Split predictions
        
    
    cv_test: np.array
        acutal ground truth observations
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
        np.ndarray
            cross validation errors for split
    '''
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = error_func(cv_test[split], cv_preds[split])
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv(cv_preds, cv_test, error_func):
    '''
    Forecast errors by forecast horizon
    
    Params:
    ------
    cv_preds: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    cv_test: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
    np.ndarray
        
    '''
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error(cv_preds[h], cv_test[h], error_func)
        horizon_errors.append(split_errors)

    return np.array(horizon_errors)

def split_coverage(cv_test, cv_intervals):
    n_splits = len(cv_test)
    cv_errors = []
        
    for split in range(n_splits):
        val = np.asarray(cv_test[split])
        lower = cv_intervals[split].T[0]
        upper = cv_intervals[split].T[1]
        
        coverage = len(np.where((val > lower) & (val < upper))[0])
        coverage = coverage / len(val)
        
        cv_errors.append(coverage)
        
    return np.array(cv_errors)
    
    
def prediction_int_coverage_cv(cv_test, cv_intervals):
    cv_test = np.array(cv_test)
    cv_intervals = np.array(cv_intervals)
    n_horizons = len(cv_test)    
    
    horizon_coverage = []
    for h in range(n_horizons):
        split_coverages = split_coverage(cv_test[h], cv_intervals[h])
        horizon_coverage.append(split_coverages)

    return np.array(horizon_coverage)  

In [29]:
def split_cv_error_scaled(cv_preds, cv_test, y_train):
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = mean_absolute_scaled_error(cv_test[split], cv_preds[split], 
                                                y_train, period=7)
        
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv_scaled(cv_preds, cv_test, y_train):
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error_scaled(cv_preds[h], cv_test[h], y_train)
        horizon_errors.append(split_errors)
        
    return np.array(horizon_errors)

In [30]:
def get_ensemble(fb_interval=0.8):
    '''
    Create and return ensemble model
    '''
    model_1 = ARIMAWrapper(order=(1,1,3), seasonal_order=(1,0,1,7), 
                           training_index=train.index,
                           holidays=new_year['ds'].tolist())

    model_2 = FbProphetWrapper(training_index=train.index, 
                               holidays=new_year)

    model_3 = ExponentialSmoothingWrapper(trend=True, damped_trend=True, 
                                          seasonal=7)
    
    estimators = {'arima': model_1, 'fbp': model_2, 'hw': model_3}
    return Ensemble(estimators, UnweightedVote())
    

# Run cross validation

This is run twices once each for 80 and 95% prediction intervals.  The 2nd run is required due to the way Prophet generates prediction intervals.

In [31]:
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
model = get_ensemble()

results = time_series_cv(model, train[REGION], val[REGION], horizons, 
                         alpha=0.2, step=7)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



# symmetric MAPE results

In [32]:
cv_preds, cv_test, cv_intervals = results
#CV point predictions smape
cv_errors = forecast_errors_cv(cv_preds, cv_test, 
                               symmetric_mean_absolute_percentage_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,2.846749,3.092036,3.182474,3.240986,3.332646,3.423800,3.494352,3.565179,3.647479,3.698958,3.747922,3.819224,4.754720
std,1.163151,1.165531,0.999269,0.803257,0.749498,0.735622,0.724623,0.677578,0.677669,0.614217,0.563291,0.619997,1.319263
min,1.117553,1.568871,2.031296,2.255147,2.462321,2.390131,2.464135,2.618192,2.736904,2.885545,2.930879,3.109268,3.257040
25%,2.010473,2.301689,2.468120,2.659297,2.766973,2.881795,3.024150,3.075994,3.275051,3.327710,3.288896,3.323892,3.581371
50%,2.498619,2.753276,2.822169,2.994488,3.259336,3.344553,3.472338,3.494913,3.623389,3.541138,3.640023,3.690660,4.754310
75%,3.645799,3.818730,3.515286,3.633556,3.776292,3.832156,3.854289,3.844016,3.834770,4.057190,4.069152,4.101410,5.385557
max,5.885374,6.085828,5.951263,5.397920,5.731698,5.805834,5.805863,5.517743,5.796322,5.595239,5.326817,5.705193,9.464877


In [33]:
#output sMAPE results to file
metric = 'smape'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-fbp-arima-hw_smape.csv


# RMSE results

In [34]:
#CV point predictions rmse
cv_errors = forecast_errors_cv(cv_preds, cv_test, root_mean_squared_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,75.587461,83.436523,88.014581,90.990705,94.457224,97.565954,100.102980,102.425431,105.039578,106.965926,108.723014,111.112421,133.053109
std,32.435023,34.347057,31.603350,28.153752,26.640948,25.537461,24.727842,23.645624,22.699520,20.362138,18.206387,18.282286,29.931394
min,34.088450,41.602797,56.882113,61.578124,68.218256,66.396401,67.429169,72.264433,74.560376,78.014230,79.384545,84.432581,99.971306
25%,52.404550,61.174895,68.334607,70.889239,74.750661,77.939530,78.826740,81.638667,86.703982,91.908781,95.446425,99.162395,105.692423
50%,68.193704,75.763037,77.083683,81.510236,83.738065,89.611431,93.119825,99.448376,102.504369,108.059575,110.213369,111.242033,134.235836
75%,89.151301,96.876296,93.049811,99.103203,106.334244,110.594108,121.285803,121.155951,117.404853,117.936319,117.721667,124.416955,147.414788
max,177.606476,191.264458,166.117575,152.960289,154.517114,153.920568,153.359584,152.769632,154.490469,148.168573,143.496467,154.755083,241.230925


In [35]:
#output RMSE to file
metric = 'rmse'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-fbp-arima-hw_rmse.csv


# Mean Absolute Scaled Error (MASE)

Scaled by one-step insample Seasonal Naive

In [36]:
#mase
cv_errors = forecast_errors_cv_scaled(cv_preds, cv_test, train[REGION])
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.769247,0.837194,0.863188,0.880267,0.906248,0.932128,0.952295,0.972063,0.994965,1.009526,1.023438,1.043465,1.291173
std,0.345692,0.348243,0.302069,0.249692,0.234744,0.229569,0.224344,0.210009,0.206947,0.186305,0.167863,0.178943,0.370061
min,0.320987,0.448615,0.570463,0.606842,0.665210,0.641020,0.661823,0.684272,0.736487,0.779424,0.757287,0.809023,0.887580
25%,0.515734,0.615694,0.676661,0.709066,0.716072,0.764096,0.783489,0.810081,0.863090,0.882178,0.909459,0.916232,0.973956
50%,0.679974,0.736956,0.758836,0.803499,0.869166,0.869367,0.935549,0.962928,0.979883,0.971366,1.004169,1.014351,1.279971
75%,0.937593,1.013996,0.928592,0.945984,1.023141,1.083264,1.102934,1.103623,1.056879,1.079097,1.113680,1.129877,1.442016
max,1.818729,1.793468,1.720280,1.555148,1.638749,1.655502,1.650945,1.567235,1.640177,1.584331,1.509627,1.613054,2.657950


In [37]:
#output mase to file.
metric = 'mase'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-fbp-arima-hw_mase.csv


# 80% Prediction Interval Coverage

In [38]:
#80% PIs
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.820106,0.812169,0.821869,0.833333,0.834921,0.836861,0.837491,0.843254,0.844209,0.849206,0.853295,0.854497,0.914054
std,0.200441,0.190171,0.157155,0.129150,0.125127,0.122425,0.108209,0.095310,0.093910,0.085286,0.077069,0.081236,0.069672
min,0.428571,0.285714,0.333333,0.428571,0.400000,0.380952,0.428571,0.500000,0.492063,0.542857,0.584416,0.559524,0.621918
25%,0.714286,0.750000,0.785714,0.767857,0.742857,0.785714,0.795918,0.812500,0.817460,0.821429,0.837662,0.845238,0.897260
50%,0.857143,0.857143,0.857143,0.892857,0.885714,0.880952,0.836735,0.839286,0.857143,0.857143,0.870130,0.857143,0.939726
75%,1.000000,0.928571,0.904762,0.928571,0.914286,0.916667,0.897959,0.910714,0.896825,0.900000,0.896104,0.898810,0.956164
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.952381,0.959184,0.964286,0.968254,0.971429,0.974026,0.976190,0.969863


In [39]:
#write 80% coverage to file
metric = 'coverage_80'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-fbp-arima-hw_coverage_80.csv


# 95% Prediction Interval Coverage

Rerun analysis and obtain 95% Prediction intervals

In [40]:
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
model = get_ensemble(fb_interval=0.95)

results = time_series_cv(model, train[REGION], val[REGION], horizons, 
                         alpha=0.05, step=7)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [41]:
#95% PIs
cv_preds, cv_test, cv_intervals = results
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.936508,0.920635,0.929453,0.937831,0.940741,0.944444,0.945578,0.947090,0.948854,0.950794,0.952381,0.952381,0.976966
std,0.127365,0.137728,0.110001,0.090020,0.078816,0.069259,0.062775,0.056574,0.052522,0.046937,0.042466,0.041503,0.010966
min,0.428571,0.428571,0.571429,0.678571,0.714286,0.761905,0.775510,0.803571,0.809524,0.828571,0.844156,0.833333,0.939726
25%,0.857143,0.892857,0.904762,0.928571,0.900000,0.892857,0.887755,0.892857,0.904762,0.914286,0.922078,0.928571,0.976712
50%,1.000000,1.000000,1.000000,0.964286,0.971429,0.976190,0.979592,0.964286,0.968254,0.971429,0.961039,0.940476,0.980822
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.992857,0.993506,0.994048,0.982192
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.986301


In [42]:
#write 95% coverage to file
metric = 'coverage_95'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-fbp-arima-hw_coverage_95.csv


# End